<a href="https://colab.research.google.com/github/henryHyun/KNU/blob/main/9%EC%A3%BC%EC%B0%A8_(15%EC%9E%A5)_%EA%B8%B0%EA%B3%84_%ED%95%99%EC%8A%B5_(1)_empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9주차 (교과서 15장) - 기계 학습

기계 학습 (machine learning) 은 기본적으로, 주어진 데이터들로부터 데이터 특성(feature)들 사이의 상관관계 및 이를 설명하는 모델을 찾아내는 방법이라는 점에서, 지난 주 수업때 다뤘던 내용과 크게 다르지는 않습니다. 다만 아래 설명할 몇 가지 차이점이 있습니다. 
* 머신 러닝은, 주어진 데이터를 가장 잘 설명하는 통계적 모델 만들기 및 통계적 유의성에 대한 검정보다는, 미지의 데이터의 성질을 주어진 특성 정보들을 바탕으로 **얼마나 잘 예측하는가**에 초점을 맞춥니다. 
* '답이 정해지지 않은' 데이터의 특성을 알아내는 것 또한 기계 학습의 주된 분야 중 하나이면, 따라서 회귀 분석 및 분류 문제보다 더 넓은 범위를 커버합니다. 

기계 학습은 상당히 방대한 분량의 범위를 포괄하는 단어이며, 따라서 매우 많은 종류의 도구 및 기법들이 있습니다. 다만, 크게 다음과 같이 분류할 수 있다고 합니다. 
* **지도 학습** (Supervised learning): 데이터와 '정답'을 주고, 여기에서 주어진 데이터를 잘 설명하는 모델을 학습하는 방법. 앞서 했던 회귀분석 (regression) 및 분류 (classification) 문제가 이 범주에 포함됩니다. 
> * ex1) 꽃잎 문제 (iris database, 분류 문제) 
>> * 특성: 꽃잎 및 꽃받침의 길이와 너비
>> * 예측하고 싶은 정보: 꽃의 품종
> * ex2) 지난 중간고사 대체 과제 (회귀분석 문제)
>> * 특성: 각 지역 내 교사 소득, 교사 1인당 학생 수, 유색인 학생 비율, ...
>> * 예측하고 싶은 정보: 학업 성취도 (~ 기준선을 통과한 학생 비율)
* **비지도 학습** (Unsupervised learning): 위 지도학습의 경우와 달리, 예측하고 싶은 정보가 (또는 '정답', '이름표') 없는 데이터들을 군집화 (clustering) 해야 할 때 사용합니다. 또는, 주어진 집단의 평균적 성질을 따르지 않는 케이스를 (anomaly, outlier) 찾아내는 경우에 또한 사용합니다. 
* **강화 학습** (Reinforced learning): 지도 학습의 한 가지 분야로도 볼 수 있으며, 데이터의 생성 및 학습이 시간에 따라 순차적으로 벌어지는 경우에 사용합니다. 알파고 등의 게임 방법 학습 방법론이 여기에 포함됩니다. 

이번 시간에는 Scikit-learn (https://scikit-learn.org/stable/) 패키지를 이용하여 지도 학습 및 비지도 학습의 예를 한 번 살펴 보기로 합시다. 지난 시간에 배웠거나 경험했던 내용들이 다른 함수를 이용하여 중복 등장할 수 있습니다. 수업 자료 외에도, 온라인에 상당히 많은 강좌들이 올라와 있기 때문에 (예를 들어 [이런 것](https://youtu.be/QO5vB7OkBCo)) 다른 강좌들을 참고해 보실 수도 있습니다. 


(강의에 사용한 코드는 아래 링크에서 받을 수 있습니다) 

Source code listings for [Numerical Python - Scientific Computing and Data Science Applications with Numpy, SciPy and Matplotlib](https://www.apress.com/us/book/9781484242452) (ISBN 978-1-484242-45-2).

In [ ]:
from sklearn import datasets
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn import tree
from sklearn import neighbors
from sklearn import svm
from sklearn import ensemble
from sklearn import cluster
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import seaborn as sns

# Built in datasets

기본적으로, scikit-learn 패키지는 연습을 위한 다양한 데이터셋들을 제공합니다. [Scikit-learn Datasets](https://scikit-learn.org/stable/datasets.html) 페이지에서 각 데이터셋들에 대한 설명 및 이를 불러오는 방법이 있습니다. 여기에서는 간단히 예로 두 가지 데이터셋을 불러 보도록 합시다. 

# Regression

과대적합 (overfitting) 의 개념을 살펴보기 위해, 그렇게 간단하지만은 않은 회귀 분석 연습을 한 번 해 보도록 하겠습니다. 이를 위해 자동으로 랜덤한 데이터셋을 만들어 주는 [make_regression](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html#sklearn.datasets.make_regression) 함수를 사용해 보도록 하겠습니다. 특성을 50개 가진, 다만 그 중에서 의미있는 특성은 10개인, 50개의 데이터를 만들어 봅시다. 

다시 한 번 선형 회귀분석을 떠올려 봅시다. $({\bf x}_i, y_i) \equiv (x_{i1}, x_{i2}, \cdots, x_{iN_{\rm features}}, y_i)$ 로 주어지는 데이터가 $N_{\rm data}$개 주어졌을 때 (${\bf x}_i$ = 특성 변수들의 모음, $y_i$ = 예측하고 싶은 목표 변수), 선형회귀분석의 목표는 다음의 에러를 최소화하는 모델 $y^{\rm pred}$ 를 만드는 것입니다. 

\begin{align}
E &= \sum^{N_{\rm data}}_{i=1} (y_i - y^{\rm pred}_{i})^2 ~~~ {\rm or} ~~~ \sum^{N_{\rm data}}_{i=1} \vert y_i - y^{\rm pred}_{i} \vert \\
y^{\rm pred}_i &\equiv \sum_{j=0}^{N_{\rm feature}} \beta_j x_{ij}
~~~~~ (x_{i0} \equiv 1)
\end{align}

50개의 데이터를 반씩 나눠서, 절반은 모델을 학습시키는데 사용하고, 나머지 절반은 이를 테스트 해 보는데 사용하겠습니다. 이는 scikit-learn의 model_selection 모듈 내 [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 함수를 통해 자동으로 이루어집니다.  

이제, [linear_model](https://scikit-learn.org/stable/modules/linear_model.html) 내 [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) 함수를 이용해 모델을 만들고 training set 을 이용해 학습을 시켜 봅시다. 

이제, 모델 학습에 사용했던 데이터에 대한 모델의 정확도와, 따로 떼어 놓았던 데이터에 학습된 모델을 적용했을 때의 정확도를 각각 살펴 봅시다. 잘 학습된 모델이라면, 양 쪽 데이터셋에 적용했을 때 정확도가 같아야 할 거라고 기대할 수 있습니다만...

model.score 함수를 통해 각각의 모델의 R-square 값을 살펴 봅시다. 

이처럼, 모델 회귀분석 (또는 '모델 학습', '모델 훈련') 에 사용한 데이터는 아주 잘 설명하지만,학습에 사용하지 않은 데이터셋에 대해 나쁜 결과를 주는 현상을 '**과대적합 (overfitting)**' 이라 합니다. 이는 기계학습을 적용하려고 할 때 가장 주의해야 할 문제입니다. 

과대적합이 발생하는 이유는 생각보다 간단한데, 풀어야 할 일차방정식의 수가 변수의 수보다 적게 주어질 때, 해당 방정식을 만족하는 해의 갯수가 무한히 많다는 것을 생각하면 과대적합이 발생하는 이유를 알기 쉽습니다. 이를 해결하는 방법은...

1. 보다 많은 데이터를 가져온다: 가장 좋은 방법이지만 그게 쉽지 않을 때가 많습니다. 
2. 학습시켜야 할 모델 파라미터 ($\beta_j$) 들에 어떤 제한 조건을 가한다: '규격화 (regularization)'

여기서는 두번째 방법의 예를 살펴 보도록 하겠습니다. 크게 두 가지 방법이 있습니다. 

1. LASSO regression: ${\rm min}_\beta \left\{ \vert X\beta - y \vert^2 + \alpha \vert \beta \vert \right\} $
2. Ridge regression:  ${\rm min}_\beta \left\{ \vert X\beta - y \vert^2 + \alpha \vert \beta \vert^2 \right\} $

여기서 $\alpha$ 는 모델 변수 $\beta$들의 성질을 결정하는, 우리가 정해야 하는 외부 변수 (이런 변수들을 '**hyperparameter**' 라고 합니다) 입니다. 보다 상세한 설명은 다음 페이지를 참고해 보세요: https://brunch.co.kr/@itschloe1/11